### 1. Import all the necessary libraries

In [1]:
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.optim as optim
%matplotlib inline

### 2. Loading the Dataset

In [3]:
df_tr = pd.read_csv("../data/train.csv")
df_tr.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


### 3. Get Computed Time from POLYLINE

Our goal is to predict the travel-time of the taxi, which can be derived from the POLYLINE length.

Recall:

```
The travel time of the trip (the prediction target of this project) is defined as the (number of points-1) x 15 seconds. 
For example, a trip with 101 data points in POLYLINE has a length of (101-1) * 15 = 1500 seconds. Some trips have missing data points in POLYLINE, indicated by MISSING_DATA column, and it is part of the challenge how you utilize this knowledge.
```


In [4]:

# Over every single 
def polyline_to_trip_duration(polyline):
  return max(polyline.count("[") - 1, 0) * 15

# This code creates a new column, "LEN", in our dataframe. The value is
# the (polyline_length - 1) * 15
df_tr["LEN"] = df_tr["POLYLINE"].apply(polyline_to_trip_duration)

### 4. Test and Extract the features: (Original Call + HR + WK + MON + TAXI_ID)

In [5]:
# Verify our guesses of the patterns of TAXI_ID such that all the IDs are in the form of 
# 20000xxx by substracting all the numbers by 20000000 and check if they are between the 
# range [0,1000).
def TAXI_ID_pattern_checker(x):
    # Test if the only last 3 digits of the TRIP_ID exhibit a pattern
    for idx in range(len(x)):
        if (x[idx]-20000000) < 0 or (x[idx]-20000000) >= 1000:
            return False
    return True

if TAXI_ID_pattern_checker(df_tr["TAXI_ID"]):
    print("Pattern is found!")

# Note that the only last three digits of the TAXI_ID are nonzero.
def parse_TAXI_ID(x):
    return (x % pow(10,3)) 

df_tr["Unique_TAXI_ID"] = df_tr["TAXI_ID"].apply(parse_TAXI_ID)

Pattern is found!


In [6]:
from datetime import datetime
def parse_time(x):
  # We are using python's builtin datetime library
  # https://docs.python.org/3/library/datetime.html#datetime.date.fromtimestamp

  # Each x is essentially a 1 row, 1 column pandas Series
  dt = datetime.fromtimestamp(x["TIMESTAMP"])
  return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

# Because we are assigning multiple values at a time, we need to "expand" our computed (year, month, day, hour, weekday) tuples on 
# the column axis, or axis 1
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
df_tr[["YR", "MON", "DAY", "HR", "WK"]] = df_tr[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

In [10]:
outlier_threshold = 3

# Trying to find relationship between "ORIGIN_CALL" and "LEN"
type_B = df_tr[df_tr["CALL_TYPE"]=="B"]

mean, std = type_B["LEN"].mean(), type_B["LEN"].std()
# "Choose all data, where the trip length is less than 3 standard deviations away from the mean"
# This is to remove outliers. Otherwise, our plots would look very squished (since there are some
# VERRRRRY long taxi trips in the dataset)
df_trimmed_B = type_B[type_B["LEN"] < mean + outlier_threshold * std]
# df_trimmed_A = df_trimmed_A[df_trimmed_A?["ORIGIN_STAND"]
# plt.scatter(df_trimmed_A["ORIGIN_CALL"], df_trimmed_A["LEN"],s=5, alpha=0.5)
# plt.xlabel("ORIGIN_CALL")
# plt.ylabel("LEN")
# plt.title("Relationship between ORIGIN_CALL and LEN")
# plt.show()

print("The correlation coefficient between ORIGIN STAND and LEN is {}".format(df_trimmed_B["ORIGIN_STAND"].corr(df_trimmed_B["LEN"])))
print("The correlation coefficient between HR and LEN is {}".format(df_trimmed_B["HR"].corr(df_trimmed_B["LEN"])))
print("The correlation coefficient between WK and LEN is {}".format(df_trimmed_B["WK"].corr(df_trimmed_B["LEN"])))
print("The correlation coefficient between MON and LEN is {}".format(df_trimmed_B["MON"].corr(df_trimmed_B["LEN"])))
print("The correlation coefficient between TAXI_ID and LEN is {}".format(df_trimmed_B["Unique_TAXI_ID"].corr(df_trimmed_B["LEN"])))

The correlation coefficient between ORIGIN STAND and LEN is -0.0566367104027852
The correlation coefficient between HR and LEN is 0.06702327616822985
The correlation coefficient between WK and LEN is -0.06274757938021086
The correlation coefficient between MON and LEN is 0.0087850062833805
The correlation coefficient between TAXI_ID and LEN is 0.00992357768658932


### 5. Data Encoding

In [15]:
outlier_threshold = 3

type_B = df_tr[df_tr["CALL_TYPE"]=="B"]

mean, std = type_B["LEN"].mean(), type_B["LEN"].std()
# "Choose all data, where the trip length is less than 3 standard deviations away from the mean"
# This is to remove outliers. Otherwise, our plots would look very squished (since there are some
# VERRRRRY long taxi trips in the dataset)
df_trimmed_B = type_B[type_B["LEN"] < mean + outlier_threshold * std]

# print(df_trimmed_A["Unique_TAXI_ID"].mean())
# print(df_trimmed_A["Unique_TAXI_ID"].std())

# print(df_trimmed_A["ORIGIN_STAND"].mean())
# print(df_trimmed_A["ORIGIN_STAND"].std())


all_features = df_trimmed_B[["ORIGIN_STAND","WK","HR"]]

# Get rid of row with nan in the ORIGIN_STAND column (or possibly other columns?)
all_features.dropna()

all_features["ORIGIN_STAND"] = all_features["ORIGIN_STAND"].astype(str)
all_features['WK'] = all_features['WK'].astype(str)
all_features['HR'] = all_features['HR'].astype(str)


all_features = pd.get_dummies(all_features)
print(f'预处理之后数据形状: {all_features.shape}')
print(all_features)

cols = list(all_features.columns.values)
print(cols)

/tmp/ipykernel_360/3533375885.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_features["ORIGIN_STAND"] = all_features["ORIGIN_STAND"].astype(str)
/tmp/ipykernel_360/3533375885.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_features['WK'] = all_features['WK'].astype(str)
/tmp/ipykernel_360/3533375885.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

预处理之后数据形状: (809308, 95)
         ORIGIN_STAND_1.0  ORIGIN_STAND_10.0  ORIGIN_STAND_11.0  \
1                       0                  0                  0   
15                      0                  0                  0   
16                      0                  0                  0   
23                      0                  0                  0   
28                      0                  0                  0   
...                   ...                ...                ...   
1710654                 0                  0                  0   
1710661                 0                  0                  0   
1710662                 0                  0                  0   
1710668                 0                  0                  0   
1710669                 0                  0                  0   

         ORIGIN_STAND_12.0  ORIGIN_STAND_13.0  ORIGIN_STAND_14.0  \
1                        0                  0                  0   
15                       0         

In [16]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 

label = df_trimmed_B["LEN"]
train_data = all_features

train_features, val_features, train_labels, val_labels = train_test_split(
    train_data, label, test_size=0.2, random_state=42)

# 训练集
train_features = torch.tensor(train_features.values, dtype=torch.float)
# 验证集
val_features = torch.tensor(val_features.values, dtype=torch.float)

train_labels = torch.tensor(train_labels.values, dtype=torch.float)
train_labels = train_labels.unsqueeze(1) 


val_labels = torch.tensor(val_labels.values, dtype=torch.float)
val_labels = val_labels.unsqueeze(1)


print(f'训练集数据: {train_features.shape}')
print(f'训练集label: {train_labels.shape}')
print(f'验证集数据: {val_features.shape}')
print(f'验证集label: {val_labels.shape}')

print(val_features)

训练集数据: torch.Size([647446, 95])
训练集label: torch.Size([647446, 1])
验证集数据: torch.Size([161862, 95])
验证集label: torch.Size([161862, 1])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [18]:
class myDataset:
    def __init__(self, data, label):
        self.data = data
        self.label = label
        
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, idx):
        return self.data[idx, :], self.label[idx]

train_dataset = myDataset(train_features, train_labels)
val_dataset = myDataset(val_features, val_labels)

# 变为迭代器
train_iter = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_iter = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=32, shuffle=False, num_workers=4)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### 6. Conversion of Dataset to Dataloader

In [19]:
import torch.nn as nn
import torch.nn.functional as F

# 初始化权重
def _weight_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight)
    elif isinstance(m, nn.BatchNorm1d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
# 网络
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(95, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)  # New layer: fc5 -> fc6
        self.fc6 = nn.Linear(64, 32)   # New layer: fc6 -> fc7
        self.fc7 = nn.Linear(32, 1)    # New layer: fc7 -> output
        self.apply(_weight_init)
        self.apply(_weight_init) # 初始化参数
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = self.fc7(x)
        return x

# 使用rmse作为自定义得分函数，这也是比赛的判定标准
def custom_score(y_true, y_pred):
#     rmse = mean_squared_error(np.log1p(y_true), np.log1p(y_pred), squared=False)
#     return rmse
    return math.sqrt(np.mean((np.array(y_pred)-np.array(y_true))*(np.array(y_pred)-np.array(y_true))))


net = Net()
# net.load_state_dict(torch.load('model.pth'))
criterion = torch.nn.MSELoss() # 损失函数为MSE
net = net.to(device) # 将网络和损失函数转化为GPU或CPU
criterion = criterion.to(device)
optimizer = torch.optim.Adam(params=net.parameters(), lr=0.005, weight_decay=0)

In [21]:
# 这是训练函数，分为train和val
# train时前向传播后向更新参数
# val时只计算损失函数
def train(net, data_iter, phase, criterion, optimizer=None):
    y_true = []
    y_pred = []
    mean_loss = []
    is_grad = True if phase == 'train' else False
    with torch.set_grad_enabled(is_grad):
        net.train()
        for step, (X, y) in enumerate(data_iter):
            X = X.to(device)
            y = y.to(device)
            out = net(X)
            loss = criterion(out, y) # 计算损失
            mean_loss.append(loss.item())
            
            if phase == 'train':
                optimizer.zero_grad() # optimizer 0
                loss.backward() # back propragation
                optimizer.step() # update the paramters

            # 将每一个step的结果加入列表，最后统一生产这一个epoch的指标  
            # 添加预测值和真实类标签
            y_pred.extend(out.detach().cpu().squeeze().numpy().tolist())
            y_true.extend(y.detach().cpu().squeeze().numpy().tolist())

    # 全量样本的rmse和平均loss
    rmse = custom_score(y_true, y_pred)
    mean_loss = np.mean(mean_loss)
    # 保留4位小数
    rmse = np.round(rmse, 4)
    mean_loss = np.round(mean_loss, 4)
    return mean_loss, rmse

In [23]:
from tqdm import tqdm
from datetime import datetime
from colorama import Fore, Back
import math

epochs = 100
loss_list_B = []
print(f'{datetime.now()} 开始训练...')
for epoch in tqdm(range(epochs)):
    train_mean_loss, train_score = train(net=net, 
                                         data_iter=train_iter, 
                                         phase='train', 
                                         criterion=criterion, 
                                         optimizer=optimizer)
    
    val_mean_loss, val_score = train(net=net, 
                                     data_iter=train_iter, 
                                     phase='val', 
                                     criterion=criterion, 
                                     optimizer=None)
    print(Fore.CYAN + Back.BLACK, end='')
    tqdm.write(f'Epoch: {epoch} Train loss: {train_mean_loss} Val loss: {val_mean_loss}', end=' ')
    tqdm.write(f'Train score: {train_score} Val score: {val_score}')
    loss_list_B.append(train_score)

print(f'{datetime.now()} 训练结束...')

2023-05-23 06:46:33.883027 开始训练...


 10%|█         | 1/10 [02:14<20:12, 134.75s/it]

Epoch: 0 Train loss: 104408.3968 Val loss: 103747.5526 Train score: 323.1221 Val score: 322.099


 20%|██        | 2/10 [04:26<17:43, 132.91s/it]

Epoch: 1 Train loss: 104243.5754 Val loss: 103510.3224 Train score: 322.866 Val score: 321.7309


...

Epoch: 7 Train loss: 103755.173 Val loss: 103579.233 Train score: 322.1104 Val score: 321.8371


 90%|█████████ | 9/10 [19:34<02:09, 129.49s/it]

Epoch: 8 Train loss: 103681.0612 Val loss: 103369.5014 Train score: 321.9951 Val score: 321.5101


100%|██████████| 10/10 [21:48<00:00, 130.85s/it]

Epoch: 9 Train loss: 103674.6041 Val loss: 103243.746 Train score: 321.986 Val score: 321.3159
2023-05-23 07:08:22.425679 训练结束...


In [ ]:
with open('loss_list_B.txt', 'w') as file:
    file.write(','.join(str(element) for element in loss_list_B))

### 8. Test set validation

In [25]:
df_tr_test = pd.read_csv("../test/test.csv")
df_tr_test.head()

df_tr_test["LEN"] = df_tr_test["POLYLINE"].apply(polyline_to_trip_duration)
df_tr_test[["YR", "MON", "DAY", "HR", "WK"]] = df_tr_test[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

df_tr_test = df_tr_test[df_tr_test["CALL_TYPE"]=="B"]

all_features_test = df_trimmed_B[["ORIGIN_STAND","WK","HR"]]

all_features_test["ORIGIN_STAND"] = all_features_test["ORIGIN_STAND"].astype(str)
all_features_test['WK'] = all_features_test['WK'].astype(str)
all_features_test['HR'] = all_features_test['HR'].astype(str)

all_features_test = pd.get_dummies(all_features_test)

missing_columns = set(all_features.columns)-set(all_features_test.columns)

for column in missing_columns:
    all_features_test[column] = 0

cols = list(all_features_test.columns.values)

prediction = net(torch.tensor(all_features_test.values, dtype=torch.float).to(device))

print("The RMSE loss against test set is:")
custom_score(prediction.tolist(),df_tr_test["LEN"].tolist())

/tmp/ipykernel_360/1882698335.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_features_test["ORIGIN_STAND"] = all_features_test["ORIGIN_STAND"].astype(str)
/tmp/ipykernel_360/1882698335.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_features_test['WK'] = all_features_test['WK'].astype(str)
/tmp/ipykernel_360/1882698335.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

The RMSE loss against test set is:


577.8904117612195

### 9. Model Saved

In [26]:
# print("Optimizer's state_dict:")
# for var_name in optimizer.state_dict():
#     print(var_name, "\t", optimizer.state_dict()[var_name])
# val_features = val_features.to(device)
torch.save(net.state_dict(), '../model/modelB.pth')